In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import src.graph_ops as go
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torch.utils import data
from sklearn.preprocessing import OneHotEncoder

In [2]:
graphs = np.load("graphs_updated.npy")
# options: graph_10+20k, graph_10+10k
labels = np.load("labels.npy")
# options: labels_10+20k, labels_10+10k

In [3]:
X = go.compress_graphs(graphs)
X = X[:,None,:]

In [4]:
first = pd.get_dummies(labels[:,0])
first[18] = 0
first[19] = 0
y = (first + pd.get_dummies(labels[:,1])).values
first = None

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [6]:
def cross_entropy(input, target, size_average=True):
    """ Cross entropy that accepts soft targets
    Args:
         pred: predictions for neural network
         targets: targets, can be soft
         size_average: if false, sum is returned instead of mean

    Examples::

        input = torch.FloatTensor([[1.1, 2.8, 1.3], [1.1, 2.1, 4.8]])
        input = torch.autograd.Variable(out, requires_grad=True)

        target = torch.FloatTensor([[0.05, 0.9, 0.05], [0.05, 0.05, 0.9]])
        target = torch.autograd.Variable(y1)
        loss = cross_entropy(input, target)
        loss.backward()
    """
    return torch.mean(-target * torch.log(input))


class LSTM(nn.Module): 
    
    #constructor
    #take in X as a parameter
    def __init__(self, X, y, hidden_dim = 50, num_layers = 4, weight_decay = 0.0005):
        super(LSTM, self).__init__()
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #self.device = 'cpu'
        
        #Find dimensionality of X, y
        X_dim = X.shape[-1]
        y_dim = y.shape[-1]
        
        # LSTM
        self.lstm = nn.LSTM(X_dim, hidden_dim, num_layers)
        
        #Output
        self.linear = nn.Linear(hidden_dim, y_dim)
        self.sigmoid = nn.Sigmoid()

        
        # Define what optimization we want to use.
        # Adam is a popular method so I'll use it.
        # L2 regularization in weight decay.
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay = weight_decay)
        self.to(self.device)
        
    # 1. input X
    def forward(self, X):
        X, _ = self.lstm(X)
        X = self.linear(X)
        X = self.sigmoid(X)
        X = 2*X/((X[:,0,:]).sum(1))[:,None,None]
        return X
    
    def loss(self, pred, true):
        #PyTorch's own cross entropy loss function.
        score = cross_entropy
        l = None
        for i in range(true.shape[1]):
            if l is None:
                l = score(pred[:,0,i], true[:,i])
            else:
                l += score(pred[:,0,i], true[:,i])
        return l
    

    def fit(self, X, y, early_stopping = True, patience = 50):
        
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y_arr = y.values
            if len(y_arr.shape) == 1:
                y_arr = y_arr[:,None]
            y_tens = Variable(torch.Tensor(y_arr).float())
        else:
            if len(y.shape) == 1:
                y = y[:,None]
            y_tens = Variable(torch.Tensor(y).float())
        
        X_tens = X_tens.to(self.device)
        y_tens = y_tens.to(self.device)
        
        if early_stopping == True:
            
            inds = np.arange(len(X_tens))
            num_train = int(len(X_tens) * .9)
            train_inds = np.random.choice(inds, num_train, replace=False)
            val_inds = np.setdiff1d(inds, train_inds)
            X_train = X_tens[train_inds]
            y_train = y_tens[train_inds]
            X_val = X_tens[val_inds]
            y_val = y_tens[val_inds]
            
        for epoch in range(1000):
            self.optimizer.zero_grad()
            if early_stopping==True:
                out_train = self.forward(X_train)
                loss = self.loss(out_train, y_train)
            if early_stopping==False:
                out_train = self.forward(X_tens)
                loss = self.loss(out_train, y_tens)
                print(loss)
                loss.backward()
                self.optimizer.step()
                continue
                
            out_val = self.forward(X_val)
            loss_val = self.loss(out_val, y_val)
            print(loss_val)


            if epoch == 0:
                min_loss = loss_val
                best_weights = self.state_dict()
                counter = 0
            else:
                if loss_val < min_loss:
                    min_loss = loss_val
                    best_weights = self.state_dict()
                    counter = 0
                else:
                    counter += 1
                    if counter == patience:
                        self.load_state_dict(best_weights)
                        return
                
            loss.backward()
            self.optimizer.step()
    
    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
            
        X_tens = X_tens.to(self.device)
        
        predict_out = self.forward(X_tens)
        temp =  predict_out.cpu().detach().numpy()
        temp[temp > 1-1e-6] = 1-1e-6
        temp[temp < 1e-6]=1e-6
        temp = temp[:,0,:]
        return temp
    
    def predict(self, X):
        proba = self.predict_proba(X).argsort(1)[:,-2:]
        proba.sort(1)
        return proba
    
    def score(self, X, y):
        y_pred = self.predict(X)
        y_true = y.argsort(1)[:,-2:]
        y_true.sort(1)
        return (y_pred == y_true).all(1).mean().item()
    
    def top_k_acc(self, X,y, k = 5):
        """Get top k accuracy for prediction. I.e., were the 2 correct nodes 
        in the top k returned. Note, k=2 is the score function."""
        y_pred = self.predict_proba(X)
        y_true = y.argsort(1)[:,-2:]  
        y_true.sort(1)
        eqs_list = []
        top_k = y_pred.argsort(1)[:,-k:]
        for i in range(y_true.shape[0]):
            row = y_true[i]
            eqs_arr = []
            for ele in row:
                eqs_arr.append(np.any(ele == top_k[i]))
            eqs_arr = np.array(eqs_arr)
            eq = eqs_arr.all()
            eqs_list.append(eq)
        eqs = np.array(eqs_list)

        return eqs.mean()

In [7]:
lstm = LSTM(X,y)

In [8]:
lstm.fit(X_train, y_train, early_stopping = True)

tensor(4.5983, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.4897, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.2801, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.9285, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.6158, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.3657, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1586, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.9943, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8836, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8212, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7890, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7685, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7506, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7332, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7191, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7158, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7271, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7442, device='cuda:0', grad_fn=<AddBack

tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5970, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5971, device='cuda:0', grad_fn=<AddBack

In [9]:
lstm.score(X_test, y_test)

0.3356

In [10]:
lstm.top_k_acc(X_test, y_test, k = 5)

0.6656